In [1]:
# macro
fileName = r'/home/liang.liang/code/Tools/python/pycharts/2019CW15.4.xlsx'
reportName = '20190411.html'
CWsMonitor = 15

TotalFeatures = 1026


In [2]:
# import lib

from pyecharts.engine import create_default_environment
from pyecharts import Page
from pyecharts import Bar, Line, Grid
from pyecharts import Liquid

import xlrd

In [3]:
data = xlrd.open_workbook(fileName)

table = data.sheets()[0]


Data =  {"KeyID":table.col_values(2,1),
         "funcGroup":table.col_values(4,1),
         "status":table.col_values(5,1),
         "group":table.col_values(6,1),
         "targetCW":table.col_values(7,1)

        }



In [4]:
# define calss for project management

In [5]:
class Features:
    """it's 37w project"""
    
    __group  = ("Audio","Media","Radio","Online","HMI","SDS","AndroidApp","AndroidOS"
                   ,"Update","Navi","System","CarConnect","DVR","PCC")    
    
    __status = ("New","In Analysis","Pending","In Implementation","In Testing","Released")
    
    
    __featureLen = len(Data["KeyID"])
    
    __autowidth = 1200
    
    __countReleased = Data["status"].count("Released")
    
    __CWs = [ ("CW%02d" %i)  for i in range(1,(CWsMonitor+1))]
    
    def __init__(self,source,export):
        self.__source = source
        self.__export = export    
        
        self.__groupBar = {}   # groupBar[team][status]
        for i in self.__group:
            self.__groupBar["%s" %i] =  {ii:0 for ii in self.__status}   

    
    
    
        for i in range(self.__featureLen):
            self.__groupBar[Data["group"][i]][Data["status"][i]] += 1
        
        
        self.__delay = {}
        for i in self.__status:
            self.__delay["%s" %i] = [0]*len(self.__group)
        
        for i in self.__status:
            groupCount = 0
            for j in self.__group:
                self.__delay[i][groupCount] = self.__groupBar[j][i]
                groupCount += 1
    
    
        self.__cwFeatures ={}
        for i in self.__status:
            self.__cwFeatures["%s" %i]=[0]*len(self.__CWs)
            
            
        for st in self.__status:
            for cw in range(0,CWsMonitor):
                cw1 = "%02d" % (cw+1)
                for i in range(0,self.__featureLen):
                    if(Data["status"][i] == st and Data["targetCW"][i] == cw1 ):
                        self.__cwFeatures[st][cw] += 1
    
    
        
        
        self.__status_group_delay = {}
        for i in self.__status:
            for j in self.__group:
                self.__status_group_delay["%s" %i] = {j:0 for j in self.__group}


        for s in self.__status:
            for g in self.__group:
                temp_count = 0
                for ii in range(len(Data["KeyID"])):
                    if (Data["status"][ii] == s and Data["group"][ii] == g ):
                        temp_count +=1
        
                self.__status_group_delay[s][g] = temp_count
    
        
    def GetExportFileName(self):
        return self.__export
        
        
    def ProjectFinishRate(self):
        liquid = Liquid(title="37W Project", title_text_size = 20
              ,subtitle = "Features Completion Rate")
        liquid.add("Liquid", [self.__countReleased/TotalFeatures],width = self.__autowidth/2)
        
        return liquid


    
    def CWStatus(self):
        barCW = Bar(title="Feature Status", title_text_size = 20
          ,subtitle ="Total items:" +str(self.__featureLen)
          ,width = self.__autowidth)

        for i in self.__status:
            barCW.add(i, self.__CWs, self.__cwFeatures[i]
            ,xaxis_rotate=0,yaxis_rotate=0,xaxis_interval= 0
            ,is_label_show=False,yaxis_pos = "left"
            ,xaxis_type = 'category',is_stack = True,is_convert = True,yaxis_label_textsize = 12
            ,yaxis_name_gap = 5
            ,is_toolbox_show = True,is_more_utils=True
            ,yaxis_interval = 0,yaxis_margin= 0
            ,legend_pos="30%"
           )
        
        return barCW
    
    
    def GroupStatus(self):
        bar1 = Bar(title="Feature Status", title_text_size = 20
          ,subtitle ="Current status for each group"
          ,width = self.__autowidth)

        for i in self.__status:
      
            bar1.add(i,self.__group, list(self.__status_group_delay[i].values())
            ,xaxis_rotate=30,yaxis_rotate=0,xaxis_interval= 0
            ,is_label_show=False,yaxis_pos = "left"
            ,xaxis_type = 'category',is_stack = True,is_convert = False,yaxis_label_textsize = 12
            ,yaxis_name_gap = 5
            ,is_toolbox_show = True,is_more_utils=True
            ,yaxis_interval = 0,yaxis_margin= 0
            ,legend_pos="30%"
           )
 
        return bar1


In [6]:
features = Features(fileName,reportName)

In [7]:
features.ProjectFinishRate()

In [8]:
#test

In [9]:
features.GroupStatus()

In [10]:
features.CWStatus()

In [11]:
#coding=utf-8
from __future__ import unicode_literals


grid1 = Grid()


grid1.add(features.ProjectFinishRate(), grid_left=  "10%")


grid2 = Grid()
grid2.add(features.GroupStatus(),grid_left=  "10%")


grid3 = Grid()
#grid3.add(barCW,grid_left=  "10%")

grid3.add(features.CWStatus(),grid_left=  "10%")



page = Page()         # step 1

page.add(grid1)
page.add(grid2)
page.add(grid3)


page.render(path = features.GetExportFileName())



In [12]:
# below is test

In [13]:
test_status = ("New","In Analysis","Pending","In Implementation","In Testing","Released")
test_group  = ("Audio","Media","Radio","Online","HMI","SDS","AndroidApp","AndroidOS"
                   ,"Update","Navi","System","CarConnect","DVR","PCC")

In [14]:
# test_groupBar = { j:{i:0} for j in test_group for i in test_status}
# test_groupBar

In [15]:
test_groupBar = {}
for i in test_group:
    test_groupBar["%s" %i] =  {i:0 for i in test_status}
test_groupBar

{'Audio': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'Media': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'Radio': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'Online': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'HMI': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'SDS': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'AndroidApp': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'AndroidOS': {'New': 0,
  'In Analysis': 0,
  'Pending': 0,
  'In Implementation': 0,
  'In Testing': 0,
  'Released': 0},
 'Update': {'New': 0,
  'In Analys

In [16]:
test_len = len(Data["KeyID"])
test_len

250

In [17]:
test_count = 0
for i in range(len(Data["KeyID"])):
    if (Data["status"][i] == "New" and Data["group"][i] == "Navi" ):
        test_count +=1

test_count

46

In [18]:
test_dic = {g:0 for g in test_group }
test_dic

{'Audio': 0,
 'Media': 0,
 'Radio': 0,
 'Online': 0,
 'HMI': 0,
 'SDS': 0,
 'AndroidApp': 0,
 'AndroidOS': 0,
 'Update': 0,
 'Navi': 0,
 'System': 0,
 'CarConnect': 0,
 'DVR': 0,
 'PCC': 0}

In [19]:
test_delay = {}
for s in test_status:
    for g in test_group:
        test_count = 0
        
#         for ii in range(len(Data["KeyID"])):
#             if (Data["status"][ii] == s and Data["group"][ii] == g ):
#                 test_count +=1
        
        test_delay["%s" %s] = {g:test_count for g in test_group}
test_delay

{'New': {'Audio': 0,
  'Media': 0,
  'Radio': 0,
  'Online': 0,
  'HMI': 0,
  'SDS': 0,
  'AndroidApp': 0,
  'AndroidOS': 0,
  'Update': 0,
  'Navi': 0,
  'System': 0,
  'CarConnect': 0,
  'DVR': 0,
  'PCC': 0},
 'In Analysis': {'Audio': 0,
  'Media': 0,
  'Radio': 0,
  'Online': 0,
  'HMI': 0,
  'SDS': 0,
  'AndroidApp': 0,
  'AndroidOS': 0,
  'Update': 0,
  'Navi': 0,
  'System': 0,
  'CarConnect': 0,
  'DVR': 0,
  'PCC': 0},
 'Pending': {'Audio': 0,
  'Media': 0,
  'Radio': 0,
  'Online': 0,
  'HMI': 0,
  'SDS': 0,
  'AndroidApp': 0,
  'AndroidOS': 0,
  'Update': 0,
  'Navi': 0,
  'System': 0,
  'CarConnect': 0,
  'DVR': 0,
  'PCC': 0},
 'In Implementation': {'Audio': 0,
  'Media': 0,
  'Radio': 0,
  'Online': 0,
  'HMI': 0,
  'SDS': 0,
  'AndroidApp': 0,
  'AndroidOS': 0,
  'Update': 0,
  'Navi': 0,
  'System': 0,
  'CarConnect': 0,
  'DVR': 0,
  'PCC': 0},
 'In Testing': {'Audio': 0,
  'Media': 0,
  'Radio': 0,
  'Online': 0,
  'HMI': 0,
  'SDS': 0,
  'AndroidApp': 0,
  'AndroidOS

In [20]:
for s in test_status:
    for g in test_group:
        test_count = 0
        
        for ii in range(len(Data["KeyID"])):
            if (Data["status"][ii] == s and Data["group"][ii] == g ):
                test_count +=1
        
        test_delay[s][g] = test_count

test_delay

list(test_delay["New"].values())

[0, 0, 0, 6, 0, 0, 10, 0, 0, 46, 0, 0, 0, 0]

In [21]:
test_statusNew_allgroup = test_delay["New"]
test_statusNew_allgroup

{'Audio': 0,
 'Media': 0,
 'Radio': 0,
 'Online': 6,
 'HMI': 0,
 'SDS': 0,
 'AndroidApp': 10,
 'AndroidOS': 0,
 'Update': 0,
 'Navi': 46,
 'System': 0,
 'CarConnect': 0,
 'DVR': 0,
 'PCC': 0}